# L16b: Deep Q-learning (DQN) Formulation Agent
In this lab, we'll look at a Deep Q-learning (DQN) agent whose objective is to learn to mix $m$ different materials to maximize the utility of the resulting mixture. Each component in the mixture has a different coefficient in the utility function, and a different unit cost.



### Tasks
Before you start, execute the `Run All Cells` command to check if you have any code or setup issues - let's get those fixed!

* __Task 1: Setup, Data, Constants__: In this task, we set up the computational environment, load the necessary packages, and prepare the `world(...)` function for our personal shopper problem. We will also define any constants we use throughout the problem set.
* __Task 2: Build the Context Models__:In this task, we will build several models of the contextual information used to inform the agent's recommendations. These models, which are [instances of the `MyBanditConsumerContextModel` type](src/Types.jl), hold various parameters that will be used in the `world(...)` function that we developed in Task 1.
* __Task 3: Evaluation of Scenarios__: In this task, we'll run different context models to evaluate how well our agent performs under various scenarios. We will use the same bandit algorithm in all cases but vary the context model to see how it influences the agent's decisions and performance. We display the results, and ask you a few discussion questions.

Tests throughout the notebook (and at the bottom section) help you determine if things are running correctly. Let's go! (Remember to answer the discussion questions.)
___

## Task 1: Setup, Data, Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. 
* The `Include.jl` file also loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem. It checks for a `Manifest.toml` file; if it finds one, packages are loaded. Other packages are downloaded and then loaded.

In [1]:
include("Include.jl");

Next, let's build the `world(...)` function. 
* The `world(...)` function takes the action vector `a::Array{Int64,1}` where the elements of `a::Array{Int64,1}` are binary variables indicating whether to select an item (`1`) or not (`0`). The length of the action vector `a` is $N$, the total number of _combinations_ available for selection. The function also takes the array `n::Array{Float64,1}` that contains the amount of each good to purchase (specified by the shopper beforehand). Finally, the `world(...)` function takes a `context` model, which encapsulates the personal shopper's environment, including the budget constraint and the penalty for exceeding it. More on the `context` in `Task 2`.

We've assumed a _linear utility function_ for the personal shopper problem, where the utility is a linear combination of the items chosen minus a penalty for exceeding the budget. The utility function $U:\mathbb{R}^{N}\rightarrow\mathbb{R}$ is defined as follows:
$$
\begin{align*}
U_{\lambda}(\mathbf{n},\mathbf{\gamma}) = \sum_{i=1}^{N} \gamma_{i}\cdot n_i - \lambda \cdot \left[\max(0, \sum_{i=1}^{N} c_i \cdot n_{i} - B)\right]^{2}
\end{align*}
$$
where $\gamma_{i}$ is the marginal utility of option $i$ (unkown to the agent, only known to the world), while the term $n_i$ denotes the amount of component $i$ in the mixture,  The quadratic penalty term is subtracted from the utility if the total cost exceeds the budget, where $c_i$ is the unit cost of item $i$.

__Hmmm__. Sometimes, we are uncertain about the benefit gained when we purchase good $i$, so let's add some randomness to the problem. In the presence of uncertainty, the utility function becomes:
$$
\begin{align*}
U_{\lambda}(\mathbf{n},\mathbf{\gamma}) = \sum_{i=1}^{N} (\gamma_{i} + \sigma_{i} \cdot Z_i) \cdot n_i - \lambda \cdot \left[\max(0, \sum_{i=1}^{N} c_i \cdot n_{i} - B)\right]^{2}
\end{align*}
$$
where $Z_i \sim \mathcal{N}(0,1)$ is a random variable drawn from a standard normal distribution for each item $i$, and $\sigma_{i}\geq{0}$ denotes the strength of the uncertainty associated with good $i$ (hyperparameter set by the shopper). This adds a stochastic element to the utility function, making it more realistic in scenarios where the benefits of purchasing items are uncertain.

In [ ]:
function world(s::Array{Float32,1}, a::Array{Float32,1}, context::MyDQNworldContextModel)

    # initialize -
    γ = context.γ; # consumer preferences (unknown to agent)
    σ = context.σ; # noise in utility calculation (unknown to agent)
    B = context.B; # max budget (unknown to agent)
    C = context.C; # unit costs of goods (unknown to agent)
    λ = context.λ; # sensitivity to the budget
    Z = context.Z; # noise model
    number_of_goods = context.m; # number of possible combinations

    # compute the reward for this choice -
    Ū = 0.0; # initial utility
    BC = 0.0; # initial budget constraint
    for i ∈ 1:number_of_goods
        
        nᵢ = s[i]; # this is the quantity purchased of good aᵢ in category i
        Cᵢ = C[i]; # cost of chosen good in category i
        γᵢ = γ[i]; # preference of good in category i
   
        # update the utility and the budget constraint -
        Ū += γᵢ*(nᵢ + σ[i,1]*rand(Z)); # compute the utility for this good, with noise. We'll use a linear utility model
        BC += nᵢ*(Cᵢ + σ[i,2]*rand(Z)); # compute the budget constraint -
    end

    # compute the utility with the budget constraint
    U = (Ū - λ*max(0.0, (BC - B))^2) .|> Float32 ; # use a penalty method to capture budget constraint

    # compute the next state -
    s′ = (s .+ (a.*s)); # update the state with the action taken to get next state

    # check: for NaN replace with zero
    s′ = max.(s′, 1e-6); # ensure that the state is non-negative
    
    # implement state constraints -
    s′ = s′ .|> Float32; # ensure that the state is non-negative
    
    # return to caller -
    return s′, -1*U; # return the next state and the reward
end;

__Constants__: Set constants we'll use in the subsequent tasks. See the comment beside the value for a description of what it is, its permissible values, etc.

In [3]:
K = 12; # TODO: Let's consider 12 different items that we need to mix together
number_of_actions = 2*K; # TODO: number of actions (2 for each item,increase/decrease)
number_of_hidden_states = 2^K; # TODO: number of hidden states (2^K made this up)
number_of_episodes = 2; # TODO: number of episodes (2^K
max_number_of_iterations = 2^14; # TODO: number of rounds for each decision task (should be geq 2^{K})
budget = 1000.0; # TODO: Budget for agent, assume 100 USD. We can change this later if we want
buffersize = 2^10; # TODO: buffer size for the agent
B = 2^6; # TODO: minibatch size for the agent

## Task 2: Setup the Context, Main, Target Networks, and the Replay Buffer
In this task, we will build several models that are required for our deep Q-learning agent. We know that the agent will have a _main_ and a _target_ network, and we will also need a replay buffer to store the agent's experiences. 

### Context Model
The context model is an instance of [the `MyDQNworldContextModel` type](src/Types.jl). It holds various parameters that will be used in the `world(...)` function that we developed in Task 1. We save our context model instance in the `contextmodel::MyDQNworldContextModel` variable.

Let's walk through what we are saying here:

In [4]:
contextmodel = let

    # initialize -
    context = nothing; # initialize the context variable to nothing; this variable will be used to store the context model
    γ = Array{Float32,1}(undef, K); # consumer preferences (unknown to agent)
    σ = Array{Float32,2}(undef, K, 2); # noise in utility calculation (unknown to agent). First col is noise for good, seccond col is noise for price
    C = Array{Float32,1}(undef, K); # unit costs of goods (unknown to agent)
    Z = Normal(0,1); # use a standard normal distribution for the noise model; this can be changed to any distribution as required
    λ = 0.0; # sensitivity to the budget constraint λ ≥ 0. If zero, then no penalty for budget constraint violation.

    # set the parameters -
    for i ∈ 1:K
        # Assigning values for γ, σ, and C for each good and price in the context model
        # For simplicity, let's assume we have K goods with equal preference
        # This can be customized as per the requirement of the simulation
        rem(i, 2) == 0 ? γ[i] = 1.0 : γ[i] = -1.0; # Fancy! if i is even, then γ[i] = 1.0, else γ[i] = -1.0       
        σ[i,:] .= 0.1; # uniform uncertainty for all goods and prices, this can be adjusted based on the specific needs of the simulation
        C[i] = 10.0 + 10.0 * (i - 1); # linearly increasing costs for goods, this can be customized as per the requirement
    end

    # TODO: Uncomment the code below to build the context model -
    # build a context model with the required parameters -
    context = build(MyDQNworldContextModel, (
        γ = γ, # consumer preferences (unknown to agent)
        σ = σ, # noise in utility calculation (unknown to agent)
        B = B, # max budget (unknown to agent)
        C = C, # unit costs of goods (unknown to agent)
        λ = λ, # sensitivity to the budget
        Z = Z, # noise model
        m = K, # number of components
    )); # build the context

    # return the model -
    context;
end;

In [5]:
# TODO: What's in the context model?

#### Main and Target Networks
The main network is used to select actions, while the target network is used to evaluate the actions taken by the main network. These networks have the same architecture but are updated at different rates. The main network is updated more frequently, while the target network is updated less frequently to provide a stable target for the Q-value updates.

__Implementation__: For the main and target models, let's build an empty model with default (random) parameter values but a fixed structure. The number and dimension of the layers and the activation functions for each layer are specified when we build the model (but we'll update the parameters during training).
* _Library_: We use [the `Flux.jl` machine learning library](https://github.com/FluxML/Flux.jl) to construct the neural network model. The model will have three layers: the input layer is a `K` $\times$ $2^{K}$ layer with [tanh activation functions](https://fluxml.ai/Flux.jl/stable/reference/models/activation/#NNlib.tanh_fast), the hidden layer is a $2^{K}$ $\times$ $\dim\mathcal{A}$ layer and the output layer is the [softmax function](https://en.wikipedia.org/wiki/Softmax_function).
* _Syntax_: The [`Flux.jl` package](https://github.com/FluxML/Flux.jl) uses some next level syntax. The model is built using [the `Chain` function](https://fluxml.ai/Flux.jl/stable/reference/models/layers/#Flux.Chain), which takes a list of layers as input. Each layer is defined using the [`Dense` type](https://fluxml.ai/Flux.jl/stable/reference/models/layers/#Flux.Dense) (in this case), which takes the number of input and output neurons as arguments. The activation function is an additional argument to [the `Dense` type](https://fluxml.ai/Flux.jl/stable/reference/models/layers/#Flux.Dense). The final layer uses [the `softmax(...)` method exported by the `NNlib.jl` package](https://fluxml.ai/NNlib.jl/dev/reference/#Softmax) to produce a probability distribution over the classes.

We save the main network in the `M::Chain` variable:

In [6]:
M = let 
    
    # TODO: Uncomment the code below to build the model!
    Flux.@layer MyFluxNeuralNetworkModel trainable=(input, hidden); # create a "namespaced" of sorts
    MyModel() = MyFluxNeuralNetworkModel( # a strange type of constructor
        Chain(
            input = Dense(K, number_of_hidden_states, tanh_fast),  # layer 1
            hidden = Dense(number_of_hidden_states, number_of_actions, tanh_fast), # layer 2
            output = NNlib.softmax) # layer 3 (output layer)
    );
    model = MyModel().chain;

    # return -
    model;
end

Chain(
  input = Dense(12 => 4096, tanh_fast),  # 53_248 parameters
  hidden = Dense(4096 => 24, tanh_fast),  # 98_328 parameters
  output = NNlib.softmax,
)                   # Total: 4 arrays, 151_576 parameters, 592.297 KiB.

The target network is a copy of the main network, but its parameters are updated less frequently. The target network is updated by copying the parameters from the main network every few episodes.

We save the target network in the `T::Chain` variable:

In [7]:
T = let

    # TODO: Uncomment the code below to build the model!
    Flux.@layer MyFluxNeuralNetworkModel trainable=(input, hidden); # create a "namespaced" of sorts
    MyModel() = MyFluxNeuralNetworkModel( # a strange type of constructor
        Chain(
            input = Dense(K, number_of_hidden_states, tanh_fast),  # layer 1
            hidden = Dense(number_of_hidden_states, number_of_actions, tanh_fast), # layer 2
            output = NNlib.softmax) # layer 3 (output layer)
    );
    model = MyModel().chain;

    # return -
    model;
end

Chain(
  input = Dense(12 => 4096, tanh_fast),  # 53_248 parameters
  hidden = Dense(4096 => 24, tanh_fast),  # 98_328 parameters
  output = NNlib.softmax,
)                   # Total: 4 arrays, 151_576 parameters, 592.297 KiB.

### Replay Buffer
The replay buffer is a data structure that stores the agent's experiences. It is used to sample random batches of experiences for training the main network. We will implement a _vanilla DQN_ agent whose replay buffer is a [simple circular buffer](https://en.wikipedia.org/wiki/Circular_buffer#:~:text=In%20computer%20science%2C%20a%20circular,easily%20to%20buffering%20data%20streams.). 
* _What goes into the buffer_? The replay buffer stores the agent's experiences in the form of tuples $(s,a,r,s^{\prime})$, where $s$ is the current state, $a$ is the action taken, $r$ is the reward received, $s^{\prime}$ is the next state.
* _Implementation_? We'll use [the circular buffer implementation exported by the `DataStructures.jl` package](https://github.com/JuliaCollections/DataStructures.jl). The [`CircularBuffer` type](https://juliacollections.github.io/DataStructures.jl/stable/circ_buffer/) is a fixed-size buffer that overwrites the oldest elements when it becomes full. The size of the buffer is set by the `buffersize::Int64`. We can generate $B$ random samples, i.e., our training minibatch from the replay buffer using [a extended `rand(...)` function](https://docs.julialang.org/en/v1/stdlib/Random/#Base.rand). 

The agent builds it's own replay buffer, so here let's play with the `CircularBuffer` type to see how it works.

In [8]:
test_replay_buffer = CircularBuffer{Int}(10); # TODO: create a circular buffer that holdes int    
for i ∈ 1:buffersize
    push!(test_replay_buffer, i); # TODO: fill the buffer with some random values
end

In [9]:
# TODO: What happnes if we push more than the buffer size?

### Learning agent model
Finally, we need to build a model for the learning agent. The agent model is an instance of [the `MyDQNLearningAgentModel` type](src/Types.jl). We save our problem model instance in the `agentmmodel::MyDQNworldProblemModel` variable.

In [10]:
agentmodel = let

    model = build(MyDQNLearningAgentModel, (
        mainnetwork = M, # the main network
        targetnetwork = T, # the target network
        number_of_actions = number_of_actions, # number of actions
        number_of_inputs = K, # number of inputs
        Δ = 0.01, # state peturbation 
    )); # build the agent

    # return -
    model
end;

What is in the agent model?

In [11]:
## TODO: make sure we understand what data our agent has

## Task 3: Let's watch the DQN agent in action.
In this task, we train the agent and then give it some random vectors and see what is says. Howvever, before we do that, let's review the training process.

#### DQN Training Algorithm
__Initialize__ the parameters of the main Q-network $Q_{\theta}(s)$ and the target Q-network $Q^{\prime}_{\theta^{-}}(s)$ to random values. Initialize a (potentially infinite) replay buffer $\mathcal{B}$. Set the hyperparameters: the learning rate $\alpha$, the discount factor $\gamma$, the exploration rate $\epsilon_{t}$, the minimum number of experiences in the replay buffer $B$, and the parameter update count $\mathcal{C}$.
- For each episode, initialize the state to $s_0$ and:
   - For each time step $t=1,\ldots,T$:
        1. Role a random number $p\in[0,1]$. If $p\leq\epsilon_{t}$, choose a random (uniform) action $a_{t}\in\mathcal{A}$. Otherwise, choose a greedy action $a_{t} = \text{arg}\max_{a\in\mathcal{A}}{Q_{\theta}(s_{t})}$.
        2. Execute action $a_{t}$, observe the reward $r_{t}$ from the _world_ and transition to the next state $s_{t+1}$. 
        3. Store the transition (experience) $\mathcal{e}=(s_t, a_t, r_t, s_{t+1})$ in the replay buffer: $\mathcal{e}\rightarrow\mathcal{B}$. 
        5. If the replay buffer $\mathcal{B}$ has a _minium number of elements_: sample a mini-batch of experiances $(s_i, a_i, r_i, s_{i+1})$ from the replay buffer.  The agent randomly samples a mini-batch of $B$ transitions from the replay buffer:  $(s_j, a_j, r_j, s_{j+1}),\, j = 1, 2, \dots, B$. Each tuple represents a state-action-reward-next state experience example collected during environment interaction.
        6. Compute the _target Q-value_ for each transition in the mini-batch using the _target Q-network_: $y_i = r_i + \gamma \cdot \max_{a^{\prime}\in\mathcal{A}}Q^{\prime}_{\theta^{-}}(s_{i+1})$ for $i=1,2,\ldots,B$.
        7. Compute the _mean squared loss_ function over the $B$ experiances collected in the mini-batch: $L(\theta) = \frac{1}{B}\sum_{i=1}^{B}\left(y_i - Q_{\theta}(s_i)\right)^2$.
        8. Perform a _single_ gradient descent step to minimize the loss function $L(\theta)$ with respect to the parameters $\theta$ of the main Q-network $Q_{\theta}(s)$: $\theta \leftarrow \theta - \alpha \nabla_{\theta}L(\theta)$, where $\alpha$ is the learning rate. 
            - _Why only a single step_? Each mini-batch is just a _small sample of the environment’s dynamics._ The goal of DQN is _online learning_: the network parameters are continuously updated as new experiences come in. If we force training to converge on each mini-batch, it risks _overfitting to that mini-batch_.
        10. Update the state $s_t \leftarrow s_{t+1}$.
        9. Every $C$ steps, update the target Q-network parameters: $\theta^{-} \leftarrow \theta$.
    - End For
- End For

We've implemented this algorithm in [the `learn(...)` method](src/Compute.jl). Let's run the training process and then test the agent on some random vectors. 

In [12]:
my_trained_agent = learn(agentmodel, world, 
    context = contextmodel, numberofepisodes = number_of_episodes, maxnumberofsteps = max_number_of_iterations, 
    minibatchsize = B, maxreplaybuffersize=buffersize);

What's in the agent's replay buffer? 

In [13]:
s, a, r, s′ = let
    
    # initialize -
    agent_replay_buffer = my_trained_agent.replaybuffer;

    # draw a random experience from the replay buffer -
    experience = rand(agent_replay_buffer); # draw a random experience
    s, a, r, s′ = experience; # unpack the experience
end;

__Check__: Do we recover the next state $s^{\prime}$ from the $s$ and $a$ values in the replay buffer?

In [14]:
let
    true_s′ = s′;
    computed_s′ = s .+ (a.*s); # compute the next state
    @assert true_s′ ≈ computed_s′; # check if the computed next state is equal to the true next state
end

In [15]:
r

-1.0f12

In [16]:
# TODO: What does a s -a-> s′ look like?
[s s′ a]

12×3 Matrix{Float32}:
 3.64695     3.64695     -0.0
 0.932298    0.932298    -0.0
 0.97093     0.97093     -0.0
 0.861048    0.861048    -0.0
 7.0f-44     7.0f-44     -0.01
 0.869832    0.869832    -0.0
 0.982602    0.982602    -0.0
 0.00026649  0.00026649  -0.0
 1.15057     1.15057     -0.0
 3.50536     3.50536     -0.0
 0.803115    0.803115    -0.0
 0.924268    0.924268    -0.0

In [17]:
let
    a = my_trained_agent.actions;
    sₒ = ones(Float32, K); # initial state
    U = my_trained_agent.mainnetwork(sₒ)
end

24-element Vector{Float32}:
 0.044361714
 0.035765365
 0.040882304
 0.040750857
 0.041108962
 0.04196192
 0.038431525
 0.042464167
 0.04384453
 0.050858583
 ⋮
 0.043478493
 0.04263552
 0.043839894
 0.04655019
 0.045217585
 0.044105347
 0.033362694
 0.037176594
 0.04507676